In [ ]:
import os
import zipfile

# Define the zip file path and extraction directory
zip_path = "/content/drive/MyDrive/Colab Notebooks/DATA.zip"
extract_dir = "/content/UTKFace"

# Extract if not already done
if not os.path.exists(extract_dir):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

print("✅ UTKFace dataset extracted successfully!")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import shutil

# Define source and target directories
src_dir = "/content/UTKFace/UTKFace"  # Change this if your dataset is elsewhere
male_dir = "/content/dataset/male"
female_dir = "/content/dataset/female"

# Ensure target directories exist
os.makedirs(male_dir, exist_ok=True)
os.makedirs(female_dir, exist_ok=True)

# Move images to correct folders
for file in os.listdir(src_dir):
    if file.endswith(".jpg"):
        parts = file.split("_")
        if len(parts) >= 2:  # Ensure filename has enough parts
            gender = parts[1]  # Gender is the second element

            src_path = os.path.join(src_dir, file)
            if gender == "0":  # FIXED: 0 is MALE
                shutil.move(src_path, os.path.join(male_dir, file))
            elif gender == "1":  # FIXED: 1 is FEMALE
                shutil.move(src_path, os.path.join(female_dir, file))

print("✅ Images sorted CORRECTLY into 'male' and 'female' folders!")

In [ ]:
!ls /content/dataset/male | wc -l  # Count male images
!ls /content/dataset/female | wc -l  # Count female images

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define dataset path
dataset_dir = "/content/dataset"

# Image Data Generator for training and validation
datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values (0-255 → 0-1)
    validation_split=0.2  # 80% training, 20% validation
)

# Load training data
train_data = datagen.flow_from_directory(
    dataset_dir,
    target_size=(128, 128),  # Resize images
    batch_size=32,
    class_mode="binary",
    subset="training"
)

# Load validation data
val_data = datagen.flow_from_directory(
    dataset_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode="binary",
    subset="validation"
)

In [ ]:
model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation="relu", input_shape=(128, 128, 3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation="relu"),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation="relu"),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),  # Prevent overfitting
    layers.Dense(1, activation="sigmoid")  # Binary classification (0=Female, 1=Male)
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

In [ ]:
history = model.fit(train_data, validation_data=val_data, epochs=10)

In [ ]:
plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.legend()
plt.show()

In [ ]:
model.save("/content/gender_model_diversifai.h5")
print("✅ Model saved successfully!")

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# Load and preprocess an image
img_path = "/content/TestPics/example8.jpg"  # Change this to any test image path
img = image.load_img(img_path, target_size=(128, 128))

# Convert image to array for prediction
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predict gender
prediction = model.predict(img_array)[0][0]
gender = "Male" if prediction > 0.5 else "Female"

# Display the image with prediction
plt.imshow(img)
plt.axis("off")  # Hide axes
plt.title(f"Predicted Gender: {gender}", fontsize=14, color="red")  # Show prediction in title
plt.show()

print(f"Predicted Gender: {gender}")

In [ ]:
import cv2
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# Load pre-trained face detector (Haar Cascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the image
img_path = "/content/Test1/Img9.jpg"
img = cv2.imread(img_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect faces
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

# Loop through detected faces
for (x, y, w, h) in faces:
    face = img[y:y+h, x:x+w]  # Crop face region
    face = cv2.resize(face, (128, 128))  # Resize for model input
    face_array = image.img_to_array(face) / 255.0
    face_array = np.expand_dims(face_array, axis=0)

    # Predict gender
    prediction = model.predict(face_array)[0][0]
    gender = "Male" if prediction > 0.5 else "Female"

    # Draw bounding box & label
    color = (0, 255, 0) if gender == "Male" else (255, 0, 0)  # Green for Male, Blue for Female
    cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
    cv2.putText(img, gender, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

# Display the result
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Gender Prediction for Multiple People", fontsize=14, color="red")
plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load pre-trained face detector (Haar Cascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the gender prediction model (replace with your model path)
model = load_model('/content/gender_model_diversifai.h5')  # Ensure you have a trained model

# Load the image
img_path = "/content/Test1/Img7.jpg"
img = cv2.imread(img_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect faces with optimized parameters
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=8, minSize=(60, 60))

# Ensure at least one face is detected
if len(faces) == 0:
    print("No faces detected.")
else:
    for (x, y, w, h) in faces:
        # Extract only the face region
        face = img[y:y+h, x:x+w]

        # Convert to RGB and resize for model input
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (128, 128))

        # Normalize and expand dimensions for prediction
        face_array = image.img_to_array(face) / 255.0
        face_array = np.expand_dims(face_array, axis=0)

        # Predict gender
        prediction = model.predict(face_array)[0][0]
        gender = "Male" if prediction > 0.5 else "Female"

        # Draw bounding box & label
        color = (0, 255, 0) if gender == "Male" else (255, 0, 0)  # Green for Male, Blue for Female
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, gender, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

# Display the result
plt.figure(figsize=(8, 6))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Face Detection with Gender Prediction", fontsize=14, color="red")
plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load a more accurate Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')

# Load the gender prediction model
model = load_model('/content/gender_model_diversifai.h5')

# Load and preprocess image
img_path = "/content/TestPics/WhatsApp Image 2025-02-12 at 10.33.04 AM.jpeg"
img = cv2.imread(img_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply CLAHE (Adaptive Histogram Equalization) to enhance face details
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
gray = clahe.apply(gray)

# Detect faces with optimized parameters
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(80, 80))

# Ensure at least one face is detected
if len(faces) == 0:
    print("No faces detected.")
else:
    for (x, y, w, h) in faces:
        # Additional filter: Ignore non-square faces (likely false positives)
        if w / h < 0.75 or w / h > 1.3:
            continue

        # Extract only the face region
        face = img[y:y+h, x:x+w]

        # Convert to RGB (ensure model compatibility)
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)

        # Resize for model input (ensure correct dimensions)
        face = cv2.resize(face, (128, 128))

        # Normalize properly based on model expectations
        face_array = image.img_to_array(face) / 255.0  # Ensure it's between 0-1
        face_array = np.expand_dims(face_array, axis=0)  # Add batch dimension

        # Predict gender
        prediction = model.predict(face_array)[0][0]
        gender = "Male" if prediction > 0.5 else "Female"

        # Draw bounding box & label
        color = (0, 255, 0) if gender == "Male" else (255, 0, 0)  # Green for Male, Blue for Female
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, gender, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

# Display the result
plt.figure(figsize=(8, 6))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Face Detection with Gender Prediction", fontsize=14, color="red")
plt.show()